In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')

In [ ]:
#!pip install pandas_profiling


In [ ]:
#import pandas_profiling

In [ ]:
#df.profile_report()

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

df['excerpt_applied_stop_words'] = df['excerpt'].apply(remove_stopwords)

df['word_count'] = df['excerpt_applied_stop_words'].apply(lambda x: len(x.replace('\n', '').split(' ')))
df['words'] = df['excerpt_applied_stop_words'].apply(lambda x: x.replace('\n', '').split(' '))

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
!pip install gensim

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [ ]:
words_list = df['words'].values.tolist()

In [ ]:
import os
from gensim.test.utils import get_tmpfile

if not os.path.exists('my_doc2vec_model'):
    
    words_list = df['words'].values.tolist()
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(words_list)]
    model = Doc2Vec(documents, vector_size=10, window=5, min_count=1, workers=4)
else:
    fname = get_tmpfile("my_doc2vec_model")
    model.save(fname)
    model = Doc2Vec.load(fname)

In [ ]:
vars(model)

In [ ]:
#model(df['excerpt_applied_stop_words'][0])

In [ ]:
vars(model.wv)

In [ ]:
model.wv.vectors

In [ ]:
import matplotlib.pyplot as plt

x = model.wv.vectors.T[0]
y = model.wv.vectors.T[1]

In [ ]:
plt.scatter(x, y)

In [ ]:
from sklearn.decomposition import PCA



X = model.wv.vectors

pca = PCA(n_components=2)
pca.fit(X)
Xd = pca.transform(X)

print(Xd)


In [ ]:
plt.scatter(Xd.T[0], Xd.T[1])

In [ ]:
import torch
import numpy as np

In [ ]:
df['vector'] = df['words'].apply(lambda x: (model.infer_vector(x).tolist()))

In [ ]:
df['vector'] = df['words'].apply(lambda x: (model.infer_vector(x).tolist()))
features = np.array(df['vector'].values.tolist()).astype(np.float32)

In [ ]:
import torch.utils.data as data_utils

df['target'].values.reshape(-1,1).shape

In [ ]:

train = data_utils.TensorDataset(torch.tensor(features), torch.tensor(df['target'].values.reshape(-1, 1).astype(np.float32)))

In [ ]:
train

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


# NOTE: pretty simple NN model
class SimpleNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(10, 100)
        self.l2 = nn.Linear(100, 200)
        self.l3 = nn.Linear(200, 50)
        self.l4 = nn.Linear(50, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.relu(self.l2(x))
        x = self.relu(self.l3(x))
        x = self.l4(x)
        return x
    

In [ ]:
torch.manual_seed(1)

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_model = SimpleNN()
print(train_model)

train_model = train_model.to(dev)

optimizer = torch.optim.Adam(train_model.parameters(), lr=0.01)

loss_func = nn.MSELoss()

In [ ]:
BATCH_SIZE = 64
EPOCH = 10000

loader = torch.utils.data.DataLoader(
    dataset=train, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

In [ ]:
from tqdm import tqdm

# FIXME: some codes have an fatal error probably because loss wouldn't decrease..
for epoch in tqdm(range(EPOCH)):
    losses = []
    for step, (b_x, b_y) in enumerate(loader):
        optimizer.zero_grad()   # clear gradients for next train
        b_x, b_y = b_x.to(dev), b_y.to(dev)
        prediction = train_model(b_x)     # input x and predict based on x
        loss = loss_func(prediction, b_y)     # must be (1. nn output, 2. target)
        losses.append(loss)
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients

    print(sum(losses) / len(losses))

In [ ]:
torch.save(train_model.state_dict(), 'model.pth')

In [ ]:
test_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')

In [ ]:
test_df

In [ ]:
test_df['excerpt_applied_stop_words'] = test_df['excerpt'].apply(remove_stopwords)

test_df['word_count'] = test_df['excerpt_applied_stop_words'].apply(lambda x: len(x.replace('\n', '').split(' ')))
test_df['words'] = test_df['excerpt_applied_stop_words'].apply(lambda x: x.replace('\n', '').split(' '))

In [ ]:
test_df

In [ ]:
test_df['vector'] = test_df['words'].apply(lambda x: (model.infer_vector(x).tolist()))
features = np.array(test_df['vector'].values.tolist()).astype(np.float32)

In [ ]:
test_df

In [ ]:
for l in test_df['vector'].values:
    print(np.array(l).astype(np.float32))

In [ ]:
test_data = torch.tensor([np.array(l).astype(np.float32) for l in test_df['vector'].values])
test_data

In [ ]:
test_result = train_model(test_data.to(dev)).reshape(1, -1)[0]
test_result.tolist()

In [ ]:
submission = pd.DataFrame({'id': test_df['id'],'target': test_result.tolist()})
submission

In [ ]:
submission.to_csv('submission.csv',index=False)